In [46]:
from active_learning.learner.standard import Learner, get_classifier
from active_learning.weak_supervision.SelfTraining import SelfTraining
import argparse
import numpy as np
import copy
import pandas as pd
import math
import random
from sklearn.metrics import accuracy_score, f1_score
from timeit import default_timer as timer
from typing import List, Dict
from active_learning.config import get_active_config
from active_learning.dataStorage import DataStorage
from active_learning.datasets import load_synthetic
from active_learning.logger import init_logger
from active_learning.merge_weak_supervision_label_strategies.MajorityVoteLabelMergeStrategy import (
    MajorityVoteLabelMergeStrategy,
)
from collections import Counter

from active_learning.weak_supervision import SyntheticLabelingFunctions
from active_learning.weak_supervision.BaseWeakSupervision import BaseWeakSupervision

import seaborn as sns
import matplotlib.pyplot as plt
#sns.set_theme(style="whitegrid")


font_size = 8

tex_fonts = {
    # Use LaTeX to write all text
    # "text.usetex": True,
    "text.usetex": False,
    "font.family": ["Times New Roman", 'sans-serif'],
    # Use 10pt font in plots, to match 10pt font in document
    "axes.labelsize": font_size,
    "font.size": font_size,
    # Make the legend/label fonts a little smaller
    "legend.fontsize": font_size,
    "xtick.labelsize": font_size,
    "ytick.labelsize": font_size,
    "xtick.bottom": True,
    "figure.autolayout": True,
}

sns.set_style("whitegrid")
sns.set_context("paper")
plt.rcParams.update(tex_fonts)  # type: ignore


# https://jwalton.info/Embed-Publication-Matplotlib-Latex/
def set_matplotlib_size(width, fraction=1):
    """Set figure dimensions to avoid scaling in LaTeX.

    Parameters
    ----------
    width: float
            Document textwidth or columnwidth in pts
    fraction: float, optional
            Fraction of the width which you wish the figure to occupy

    Returns
    -------
    fig_dim: tuple
            Dimensions of figure in inches
    """
    # Width of figure (in pts)
    fig_width_pt = width * fraction

    # Convert from pt to inches
    inches_per_pt = 1 / 72.27

    # Golden ratio to set aesthetic figure height
    # https://disq.us/p/2940ij3
    golden_ratio = (5 ** 0.5 - 1) / 2

    # Figure width in inches
    fig_width_in = fig_width_pt * inches_per_pt
    # Figure height in inches
    fig_height_in = fig_width_in * golden_ratio

    fig_dim = (fig_width_in, fig_height_in)

    return fig_dim


#width = 505.89
width = 1500


df = pd.read_csv("~/exp_results/run_how_to/exp_results.csv")
df

,n_samples,n_features,n_informative,n_redundant,n_repeated,n_classes,n_clusters_per_class,weights,flip_y,class_sep,...,amount_of_lastly_al_samples,JOB_ID,AL_SAMPLES_WEIGHT,AMOUNT_OF_LFS,DATASET,DATASET_RANDOM_GENERATION_SEED,FRACTION_OF_INITIALLY_LABELLED_SAMPLES,FRACTION_OF_LASTLY_AL_LABELLED_SAMPLES,LF_RANDOM_SEED,MERGE_WS_SAMPLES_STRATEGY
0,303,14,14,0,0,2,?,?,?,?,...,35,75,29,1.215732,HEART,28296,0.374271,0.370642,238068,MajorityVoteLabelMergeStrategy
1,194,49,49,0,0,8,?,?,?,?,...,7,100,6,3.108981,flag,628862,0.633401,0.240146,771955,RandomLabelMergeStrategy
2,306,4,4,0,0,2,?,?,?,?,...,85,94,34,2.993717,HABERMAN,618363,0.307777,0.816386,860396,MajorityVoteLabelMergeStrategy
3,178,14,14,0,0,3,?,?,?,?,...,5,0,52,8.927180,wine,644168,0.598658,0.156019,999891,RandomLabelMergeStrategy
4,306,4,4,0,0,2,?,?,?,?,...,75,29,36,1.038934,HABERMAN,751103,0.226496,0.645173,214836,MajorityVoteLabelMergeStrategy
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9334,48842,105,105,0,0,4,?,?,?,?,...,9706,9245,50,4.401298,adult,869522,0.533442,0.852170,721962,SnorkelLabelMergeStrategy
9335,10000,3073,3073,0,0,10,?,?,?,?,...,2994,9110,65,2.502163,cifar10,673335,0.399166,0.999769,280095,MajorityVoteLabelMergeStrategy
9336,195,23,23,0,0,2,?,?,?,?,...,8,9103,33,4.402870,parkinsons,616999,0.229822,0.102335,907849,MajorityVoteLabelMergeStrategy
9337,1000,25,25,0,0,2,?,?,?,?,...,214,9107,39,1.402683,GERMAN,494724,0.485892,0.836036,859891,MajorityVoteLabelMergeStrategy


In [47]:
df.keys()#

Index(['n_samples', 'n_features', 'n_informative', 'n_redundant', 'n_repeated',
       'n_classes', 'n_clusters_per_class', 'weights', 'flip_y', 'class_sep',
       'hypercube', 'scale', 'random_state', 'ABSTAIN_THRESHOLDS',
       'LF_CLASSIFIERS', 'AMOUNT_OF_LF_FEATURESSS', 'f1_initial',
       'acc_initial', 'f1_ws', 'acc_ws',
       'acc_ws_and_al_UncertaintyMaxMargin_no_ws',
       'f1_ws_and_al_UncertaintyMaxMargin_no_ws',
       'acc_al_and_al_UncertaintyMaxMargin_no_ws',
       'f1_al_and_al_UncertaintyMaxMargin_no_ws',
       'acc_ws_and_al_UncertaintyMaxMargin_with_ws',
       'f1_ws_and_al_UncertaintyMaxMargin_with_ws',
       'acc_al_and_al_UncertaintyMaxMargin_with_ws',
       'f1_al_and_al_UncertaintyMaxMargin_with_ws', 'acc_ws_and_al_Random',
       'f1_ws_and_al_Random', 'acc_al_and_al_Random', 'f1_al_and_al_Random',
       'acc_ws_and_al_CoveredByLeastAmountOfLf',
       'f1_ws_and_al_CoveredByLeastAmountOfLf',
       'acc_al_and_al_CoveredByLeastAmountOfLf',
       'f

In [115]:
def hist_plot(df: pd.DataFrame, COMPARE_KEYS:List[str], TITLE:str=None, OUTPUT_PATH: str=None, LABEL:str="AUC-F1 (\\%)"):
    fig = plt.figure(figsize=set_matplotlib_size(width, fraction=0.5))

    # fig = plt.gcf()  # type: ignore
    # fig.set_size_inches(18.5, 10.5)

    selection_list = []
    for key in COMPARE_KEYS:
        selection_list.append((df[key], key))
    #for strategy in df["strategy"].unique():
    #    selection_list.append((df[df["strategy"] == strategy]["f1_auc"], strategy))

    for selection, label in sorted(selection_list, key=lambda tup: tup[1]):
        #  ax = sns.distplot(selection, label=label, **kwargs)
        #  label = "{}-{:>3}: {:.4g}% ".format(label[0], label[1], selection.mean() * 100)
        label = "{:<10}{:>5.4g}% ".format(label + ":", selection.mean() * 100)
        ax = sns.kdeplot(selection, label=label)
        # ax = sns.distplot(selection, label=label, kde=True)
        #ax.set_xlim(0, 1)
        ax.set_xlim(0.5, 0.6)
        #  ax.set_xlim(0.75, 0.8)
        #  ax.set_xlim(80, 875)

        mean = selection.mean()
        if selection.count() == 0:
            low = high = mean
        else:
            low = selection.mean() - 1.96 * selection.std() / math.sqrt(
                selection.count()
            )
            high = selection.mean() + 1.96 * selection.std() / math.sqrt(
                selection.count()
            )
        print(label)
        ax.axvline(mean, color=plt.gca().lines[-1].get_color())  # type: ignore
        ax.axvspan(low, high, alpha=0.2, color=plt.gca().lines[-1].get_color())  # type: ignore

        ax.set_xticks([x for x in ax.get_xticks()]) # stupid "workaround" for https://github.com/matplotlib/matplotlib/issues/18848
        ax.set_xticklabels(["{:.0%}\\%".format(x) for x in ax.get_xticks()])
    ax.set_title(TITLE)  # type: ignore
    # ax.set_ylabel("")
    ax.set_xlabel(LABEL)  # type: ignore
    ax.legend(  # type: ignore
        loc="center left"
    )  # , bbox_to_anchor=(0.0, 1.01), borderaxespad=0, frameon=False, ncol=len(selection))  # type: ignore
    # ax.xa

def comparison_table(df: pd.DataFrame, COMPARE_KEYS:List[str], TITLE:str=""):
    results = []
    for key in COMPARE_KEYS:
        selection = df[key]

        mean = selection.mean()
        error_area = 1.96 * selection.std() / math.sqrt(
                selection.count()
            )
        results.append([key, mean, error_area])
    print(TITLE)
    print("{:>60} {:>6} {:>4}".format("", "Mean", ""))
    for result in results:
        print("{:<50} {:>6.2%} ±{:>4.2%}".format(result[0], float(result[1]), float(result[2])))

def answer_questions(df: pd.DataFrame, COMPARE_KEYS:List[str], metric):
    results = {}
    for key in COMPARE_KEYS:
        selection = df[key]

        mean = selection.mean()
        error_area = 1.96 * selection.std() / math.sqrt(
                selection.count()
            )
        results[key] = (mean, error_area)
    
    better_than_initial = [k for k,v in results.items() if v[0] > results[metric + "_initial"][0]]
    better_than_initial_with_ws = [k for k in better_than_initial if "ws_and_al" in k]
    print("Which are better than initial?", better_than_initial)
    print("Which are better than initial, with WS?", better_than_initial_with_ws)

    al_ws_al_better_than_al_al = []

    for k in results.keys():
        if k.startswith(metric + "_ws_and_al"):
            if results[k][0] > results[k.replace("_ws_and_al", "_al_and_al")][0]:
                al_ws_al_better_than_al_al.append(k)

    print("Where is al+ws+al better than al+al?", al_ws_al_better_than_al_al)
    
    #print("Where is the mean improvement within the 95% error range?")
    
    
# 01 general improvement
for metric in ['acc', 'f1']:
    COMPARE_KEYS = [metric+"_initial", metric +"_ws"]
    for AL_SAMPLING_STRATEGY in ["UncertaintyMaxMargin_no_ws",
            "UncertaintyMaxMargin_with_ws",
            "Random",
            "CoveredByLeastAmountOfLf",
            "ClassificationIsMostWrong",
            "GreatestDisagreement"]:
            COMPARE_KEYS.append(metric + '_ws_and_al_'+AL_SAMPLING_STRATEGY)
            COMPARE_KEYS.append(metric + '_al_and_al_'+AL_SAMPLING_STRATEGY)
    comparison_table(df, COMPARE_KEYS=COMPARE_KEYS, TITLE="General Improvement after AL + WS " + metric)
    answer_questions(df, COMPARE_KEYS=COMPARE_KEYS, metric=metric)

General Improvement after AL + WS acc
                                                               Mean     
acc_initial                                        58.06% ±0.45%
acc_ws                                             56.00% ±0.44%
acc_ws_and_al_UncertaintyMaxMargin_no_ws           57.71% ±0.45%
acc_al_and_al_UncertaintyMaxMargin_no_ws           58.61% ±0.46%
acc_ws_and_al_UncertaintyMaxMargin_with_ws         57.71% ±0.45%
acc_al_and_al_UncertaintyMaxMargin_with_ws         58.61% ±0.46%
acc_ws_and_al_Random                               57.59% ±0.44%
acc_al_and_al_Random                               58.21% ±0.46%
acc_ws_and_al_CoveredByLeastAmountOfLf             57.17% ±0.45%
acc_al_and_al_CoveredByLeastAmountOfLf             58.16% ±0.46%
acc_ws_and_al_ClassificationIsMostWrong            56.98% ±0.44%
acc_al_and_al_ClassificationIsMostWrong            57.50% ±0.46%
acc_ws_and_al_GreatestDisagreement                 57.13% ±0.44%
acc_al_and_al_GreatestDisagreement          

In [124]:


# group by stuff
for group_by in ['DATASET', 'MERGE_WS_SAMPLES_STRATEGY']:
    print()
    print("#"*80)
    print(group_by)
    print("#"*80)
    for group_by_value in df[group_by].unique():
        print()
        print(group_by_value)
        for metric in ['acc', 'f1']:
            COMPARE_KEYS = [metric+"_initial", metric +"_ws"]
            for AL_SAMPLING_STRATEGY in ["UncertaintyMaxMargin_no_ws",
                    "UncertaintyMaxMargin_with_ws",
                    "Random",
                    "CoveredByLeastAmountOfLf",
                    "ClassificationIsMostWrong",
                    "GreatestDisagreement"]:
                    COMPARE_KEYS.append(metric + '_ws_and_al_'+AL_SAMPLING_STRATEGY)
                    COMPARE_KEYS.append(metric + '_al_and_al_'+AL_SAMPLING_STRATEGY)
            #comparison_table(df, COMPARE_KEYS=COMPARE_KEYS, TITLE="General Improvement after AL + WS " + metric)
            answer_questions(df[df[group_by] == group_by_value], COMPARE_KEYS=COMPARE_KEYS, metric=metric)


################################################################################
DATASET
################################################################################

HEART
Which are better than initial? ['acc_ws', 'acc_ws_and_al_UncertaintyMaxMargin_no_ws', 'acc_ws_and_al_UncertaintyMaxMargin_with_ws', 'acc_ws_and_al_Random', 'acc_ws_and_al_ClassificationIsMostWrong', 'acc_ws_and_al_GreatestDisagreement']
Which are better than initial, with WS? ['acc_ws_and_al_UncertaintyMaxMargin_no_ws', 'acc_ws_and_al_UncertaintyMaxMargin_with_ws', 'acc_ws_and_al_Random', 'acc_ws_and_al_ClassificationIsMostWrong', 'acc_ws_and_al_GreatestDisagreement']
Where is al+ws+al better than al+al? ['acc_ws_and_al_UncertaintyMaxMargin_no_ws', 'acc_ws_and_al_UncertaintyMaxMargin_with_ws', 'acc_ws_and_al_Random', 'acc_ws_and_al_CoveredByLeastAmountOfLf', 'acc_ws_and_al_ClassificationIsMostWrong', 'acc_ws_and_al_GreatestDisagreement']
Which are better than initial? ['f1_ws', 'f1_ws_and_al_UncertaintyMaxMargi

In [125]:
for discrete_group_by in ['AMOUNT_OF_LFS', 'FRACTION_OF_INITIALLY_LABELLED_SAMPLES', 'FRACTION_OF_LASTLY_AL_LABELLED_SAMPLES', 'AL_SAMPLES_WEIGHT', 'amount_of_initial_al_samples', 'amount_of_lastly_al_samples', 'AMOUNT_OF_LF_FEATURESSS']:
    print(discrete_group_by)


AMOUNT_OF_LFS
FRACTION_OF_INITIALLY_LABELLED_SAMPLES
FRACTION_OF_LASTLY_AL_LABELLED_SAMPLES
AL_SAMPLES_WEIGHT
amount_of_initial_al_samples
amount_of_lastly_al_samples
AMOUNT_OF_LF_FEATURESSS
